In [103]:
from catboost import CatBoostClassifier
import pandas as pd

In [104]:
df = pd.read_csv("DATA/new_test.csv")
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod,numeric_ans
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,0,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,...,2018-02-25 07:56:00,0,0,0,0,210.750000,138.666667,0,238,0
1,14804,Транспортный шлюз,Сервис61,1,1,1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,...,2018-02-24 07:54:00,1,1,1,1,186.683333,121.383333,1,644,0
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,...,2018-03-23 07:56:00,3,0,2,2,834.716667,1025.383333,0,238,0
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,0,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,...,NaN,3,0,3,3,-1.000000,121.250000,0,238,0
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,...,2018-03-23 07:56:00,3,0,2,2,834.700000,1004.366667,0,238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,47,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,...,2018-03-14 07:53:00,3,1,90,3,394.950000,233.666667,3,177,0
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,47,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,...,2018-03-14 07:53:00,1,1,8,3,394.916667,233.133333,1,176,0
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,47,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,...,2018-03-14 07:53:00,0,1,10,3,394.800000,279.516667,0,164,0
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,121,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,...,2018-03-14 10:43:00,3,1,108,3,397.450000,279.333333,3,184,0


In [105]:
model = CatBoostClassifier()
model.load_model("DATA/issue_model")


In [106]:
test = df[["numeric_sod", "numeric_ans", 'Тип обращения на момент подачи', 'Приоритет',
            'Влияние', 'range_time', 'time_to_solve', 'Система', 'Место',
            'Функциональная группа', 'koef']]
test

,numeric_sod,numeric_ans,Тип обращения на момент подачи,Приоритет,Влияние,range_time,time_to_solve,Система,Место,Функциональная группа,koef
0,238,0,0,0,0,210.750000,138.666667,0,0,0,0
1,644,0,0,1,1,186.683333,121.383333,1,1,1,1
2,238,0,0,0,0,834.716667,1025.383333,2,2,0,0
3,238,0,0,0,0,-1.000000,121.250000,3,3,0,0
4,238,0,0,0,0,834.700000,1004.366667,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6339,177,0,1,0,1,394.950000,233.666667,90,3,47,3
6340,176,0,1,1,1,394.916667,233.133333,8,3,47,1
6341,164,0,1,0,1,394.800000,279.516667,10,3,47,0
6342,184,0,1,0,1,397.450000,279.333333,108,3,121,3


In [107]:
ans = model.predict(test)
ans

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [108]:
df['Тип обращения итоговый'] = ans
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod,numeric_ans
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,0,2018-02-22 07:51:00,2018-02-16 13:11:00,0,0,...,2018-02-25 07:56:00,0,0,0,0,210.750000,138.666667,0,238,0
1,14804,Транспортный шлюз,Сервис61,1,1,1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,0,...,2018-02-24 07:54:00,1,1,1,1,186.683333,121.383333,1,644,0
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-31 06:36:00,2018-02-16 13:13:00,0,0,...,2018-03-23 07:56:00,3,0,2,2,834.716667,1025.383333,0,238,0
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,0,2018-02-21 14:29:00,2018-02-16 13:14:00,0,0,...,NaN,3,0,3,3,-1.000000,121.250000,0,238,0
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-30 09:36:00,2018-02-16 13:14:00,0,0,...,2018-03-23 07:56:00,3,0,2,2,834.700000,1004.366667,0,238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,47,2018-03-07 14:36:00,2018-02-25 20:56:00,1,0,...,2018-03-14 07:53:00,3,1,90,3,394.950000,233.666667,3,177,0
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,47,2018-03-07 14:06:00,2018-02-25 20:58:00,1,0,...,2018-03-14 07:53:00,1,1,8,3,394.916667,233.133333,1,176,0
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,47,2018-03-09 12:36:00,2018-02-25 21:05:00,1,0,...,2018-03-14 07:53:00,0,1,10,3,394.800000,279.516667,0,164,0
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,121,2018-03-09 12:36:00,2018-02-25 21:16:00,1,0,...,2018-03-14 10:43:00,3,1,108,3,397.450000,279.333333,3,184,0


In [109]:
df[df['Статус'] == 0]

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod,numeric_ans
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,0,2018-02-21 14:29:00,2018-02-16 13:14:00,0,0,...,NaN,3,0,3,3,-1.0,121.250000,0,238,0
124,14927,Консультация,Сервис7,1,0,18,2018-02-22 07:06:00,2018-02-16 14:51:00,0,0,...,NaN,1,0,13,49,-1.0,136.250000,0,281,0
164,14967,Замена/добавление Сертификат на сервере Система13,Сервис13,1,0,0,2018-02-22 07:26:00,2018-02-16 15:11:00,0,0,...,NaN,1,0,2,44,-1.0,136.250000,0,221,0
234,15037,Замена/добавление Сертификат на сервере Система13,Сервис13,1,0,0,2018-02-22 07:51:00,2018-02-17 00:34:00,0,0,...,NaN,1,0,2,68,-1.0,127.283333,0,221,0
278,15081,Изменение атрибутов учётной записи,Сервис7,1,0,0,2018-02-22 07:51:00,2018-02-17 04:18:00,0,0,...,NaN,1,0,13,88,-1.0,123.550000,0,276,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6110,20913,Разблокировать учетную запись пользователя Сис...,Сервис21,1,0,42,2018-03-09 13:42:00,2018-02-25 12:27:00,0,0,...,NaN,1,0,29,50,-1.0,289.250000,0,505,0
6128,20931,Консультация или решение проблем связанных с с...,Сервис13,1,0,353,2018-03-09 14:01:00,2018-02-25 12:46:00,0,0,...,NaN,1,1,2,13,-1.0,289.250000,1,282,0
6216,21019,Отключение учётной записи,Сервис7,1,0,0,2018-03-09 15:28:00,2018-02-25 14:13:00,0,0,...,NaN,1,0,13,106,-1.0,289.250000,0,335,0
6221,21024,Консультация,Сервис7,1,0,356,2018-03-09 15:34:00,2018-02-25 14:19:00,0,0,...,NaN,1,0,43,13,-1.0,289.250000,0,281,0


In [110]:
for index, row in df.iterrows():
    if row[8] != row[9] and row[4] == 0:
        df.loc[index, 'Тип обращения итоговый'] = row[8]



In [111]:
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod,numeric_ans
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,0,2018-02-22 07:51:00,2018-02-16 13:11:00,0,0,...,2018-02-25 07:56:00,0,0,0,0,210.750000,138.666667,0,238,0
1,14804,Транспортный шлюз,Сервис61,1,1,1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,0,...,2018-02-24 07:54:00,1,1,1,1,186.683333,121.383333,1,644,0
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-31 06:36:00,2018-02-16 13:13:00,0,0,...,2018-03-23 07:56:00,3,0,2,2,834.716667,1025.383333,0,238,0
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,0,2018-02-21 14:29:00,2018-02-16 13:14:00,0,0,...,NaN,3,0,3,3,-1.000000,121.250000,0,238,0
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-30 09:36:00,2018-02-16 13:14:00,0,0,...,2018-03-23 07:56:00,3,0,2,2,834.700000,1004.366667,0,238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,47,2018-03-07 14:36:00,2018-02-25 20:56:00,1,0,...,2018-03-14 07:53:00,3,1,90,3,394.950000,233.666667,3,177,0
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,47,2018-03-07 14:06:00,2018-02-25 20:58:00,1,0,...,2018-03-14 07:53:00,1,1,8,3,394.916667,233.133333,1,176,0
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,47,2018-03-09 12:36:00,2018-02-25 21:05:00,1,0,...,2018-03-14 07:53:00,0,1,10,3,394.800000,279.516667,0,164,0
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,121,2018-03-09 12:36:00,2018-02-25 21:16:00,1,0,...,2018-03-14 10:43:00,3,1,108,3,397.450000,279.333333,3,184,0


In [112]:
def get_requalification_type(ticketStartType, ticketFinalType):
    if ticketStartType == ticketFinalType:
        return 0
    return 2 if ticketStartType == "Инцидент" else 1
map_type = {0: 'Запрос', 1: "Инцидент"}

In [113]:
get_requalification_type("Инцидент", "Инцидент")

0

In [114]:
submission = df[['id', 'Тип обращения на момент подачи', 'Тип переклассификации', 'Тип обращения итоговый', ]].copy()
submission['Тип обращения на момент подачи'] = submission['Тип обращения на момент подачи'].map(map_type)
submission['Тип обращения итоговый'] = submission['Тип обращения итоговый'].map(map_type)
submission['Тип переклассификации'] = submission.apply(lambda row: get_requalification_type(row['Тип обращения на момент подачи'], row['Тип обращения итоговый']), axis=1)
submission

,id,Тип обращения на момент подачи,Тип переклассификации,Тип обращения итоговый
0,14803,Запрос,0,Запрос
1,14804,Запрос,0,Запрос
2,14805,Запрос,0,Запрос
3,14806,Запрос,0,Запрос
4,14807,Запрос,0,Запрос
...,...,...,...,...
6339,21142,Инцидент,2,Запрос
6340,21143,Инцидент,2,Запрос
6341,21144,Инцидент,2,Запрос
6342,21145,Инцидент,2,Запрос


In [115]:
submission['Тип переклассификации'].value_counts()

0    6020
2     322
1       2
Name: Тип переклассификации, dtype: int64

In [116]:
len(submission)

6344

In [117]:
submission.drop(['Тип обращения на момент подачи'], axis=1).to_csv("submission.csv", index=False)